In [1]:
import os
import cv2
import random
#import torch
#import torchvision
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
#import torch.utils.data as data
from PIL import Image
from tensorflow import keras
#from torch.utils.data import DataLoader
from OfficialWork.SIOA.WheatClassification.SourceCode.DataInitialize import DataInitialize

In [2]:
data_initializer = DataInitialize(root_path=r'E:\OfficialWorkRemote\OfficialWork\SIOA\WheatClassification\SourceData',
                           train_test_rate=0.6,reshape_size=(128, 128), mode='train')

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride_length=1,padding='SAME'):
    return tf.nn.conv2d(x, W, strides=[1, stride_length, stride_length, 1], padding=padding)

def max_pool_2x2(x,pool_size=2,stride_length=2,padding='SAME'):
    return tf.nn.max_pool(x, ksize=[1, pool_size, pool_size, 1],
                          strides=[1, stride_length, stride_length, 1], padding=padding)

def cnn_build(data_initializer):
    sess = tf.InteractiveSession()
    x_image = tf.placeholder('float', shape=[None, 128,128,1])
    y_true = tf.placeholder('float', shape=[None, 2])
    # 第一层卷积
    filter_conv1 = weight_variable([9, 9, 1, 16])
    bias_conv1 = bias_variable([16])
    h_conv1 = tf.nn.relu(conv2d(x_image, filter_conv1) + bias_conv1)
    print(h_conv1.shape)  # (?, 128, 128, 16)
    # 池化
    h_pool1 = max_pool_2x2(h_conv1)
    print(h_pool1.shape)  # (?, 64, 64, 16)
    # 第二层卷积
    filter_conv2 = weight_variable([9, 9, 16, 32])
    bias_conv2 = bias_variable([32])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, filter_conv2) + bias_conv2) # (?, 64, 64, 32)
    # 池化
    h_pool2 = max_pool_2x2(h_conv2) # (?, 32, 32, 32)
    # 第三层卷积
    filter_conv3 = weight_variable([5, 5, 32, 64])
    bias_conv3 = bias_variable([64])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, filter_conv3) + bias_conv3) # (?, 32, 32, 64)
    # 池化
    h_pool3 = max_pool_2x2(h_conv3)  #  (?, 16, 16, 64)
    # 第四层卷积
    filter_conv4 = weight_variable([3, 3, 64, 128])
    bias_conv4 = bias_variable([128])
    h_conv4 = tf.nn.relu(conv2d(h_pool3, filter_conv4) + bias_conv4)  # (?, 16, 16, 128)
    # 将第四层卷积层舒展
    h_conv4_flat = tf.reshape(h_conv4, [-1, 16*16*128])
    # 全连接层1
    W_fc1 = weight_variable([16 * 16 * 128, 1024])
    b_fc1 = bias_variable([1024])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv4_flat, W_fc1) + b_fc1)  # (?, 1024)
    # 全连接层2
    W_fc2 = weight_variable([1024, 512])
    b_fc2 = bias_variable([512])
    h_fc2=tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)  # (?, 512)
    # dropout 层
    rate = tf.placeholder("float")
    h_fc2_drop = tf.nn.dropout(h_fc2, rate=rate)
    # softmax 输出层y_conv
    W_fc3 = weight_variable([512, 2])
    b_fc3 = bias_variable([2])
    y_conv = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)  # (?, 2)
    # 模型训练
    cross_entropy = -tf.reduce_sum(y_true*tf.log(y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_true,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        data_initializer.mode = 'train'
        train_batch = data_initializer.data_generator(30)

        r_channel = np.reshape(train_batch[0][:,0,:,:], [-1,128,128,1])
        train_step.run(feed_dict={x_image:r_channel, y_true: train_batch[1],rate: 0.8})
        train_accuracy = accuracy.eval(feed_dict={x_image:r_channel, y_true: train_batch[1],rate: 0.8}) #keep_prob: 1.0
        print("step %d, training accuracy %g"%(i, train_accuracy))
        #data_initializer.mode = 'test'
        #test_batch = data_initializer.data_generator(30)
        #r_channel = np.reshape(test_batch[0][:,0,:,:], [-1,128,128,1])
        #print("test accuracy %g"%accuracy.eval(feed_dict={x_image:r_channel, y_true: test_batch[1],rate: 0.8}))

In [4]:
cnn_build(data_initializer)

Instructions for updating:
Colocations handled automatically by placer.
(?, 128, 128, 16)
(?, 64, 64, 16)
step 0, training accuracy 0.533333
step 1, training accuracy 0.5
step 2, training accuracy 0.466667
step 3, training accuracy 0.6
step 4, training accuracy 0.533333
step 5, training accuracy 0.5
step 6, training accuracy 0.533333
step 7, training accuracy 0.433333
step 8, training accuracy 0.5
step 9, training accuracy 0.533333
step 10, training accuracy 0.5
step 11, training accuracy 0.533333
step 12, training accuracy 0.466667
step 13, training accuracy 0.566667
step 14, training accuracy 0.633333
step 15, training accuracy 0.433333
step 16, training accuracy 0.466667
step 17, training accuracy 0.5
step 18, training accuracy 0.633333
step 19, training accuracy 0.533333
step 20, training accuracy 0.466667
step 21, training accuracy 0.6
step 22, training accuracy 0.566667
step 23, training accuracy 0.466667
step 24, training accuracy 0.633333
step 25, training accuracy 0.533333
ste